In [2]:
import pandas as pd
import os
import numpy as np

#file = ('scraped_data.csv')
#df = pd.read_csv(file) 
#df.dropna()

train = pd.read_csv('train_symp.csv')
train.dropna()


,Unnamed: 0,Disease,Symptoms
0,445,Atopic Dermatitis,Symptoms of keratosis pilaris may include:You ...
1,44,Atopic Dermatitis,"Keratosis pilaris can occur at any age, but it..."
2,45,Atopic Dermatitis,Actinic keratoses vary in appearance. Symptoms...
3,404,Atopic Dermatitis,Symptoms of seborrheic keratosis are skin grow...
4,403,Atopic Dermatitis,Symptoms may include:Small bumps that look lik...
...,...,...,...
326,184,Seborrheic Keratoses and other Benign Tumors,"On an earlobe, youll likely see a round, soli..."
327,161,Seborrheic Keratoses and other Benign Tumors,A keratoacanthoma appears and grows rapidly ov...
328,330,Seborrheic Keratoses and other Benign Tumors,Feeling a lump just beneath the skin\nIt may b...
329,329,Seborrheic Keratoses and other Benign Tumors,The growths can:\n\nBe slightly raised from th...


# BERT MODEL

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Convert symptoms column to a list
texts = train["Symptoms"].tolist()

# Convert your disease column to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(train["Disease"].tolist())

# Split data into train and validation sets
#train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# For now, train and validation sets are the same
train_texts = texts
val_texts = texts
train_labels = list(labels)
val_labels = list(labels)

train_texts = [str(text) for text in train_texts if text is not None]
val_texts = [str(text) for text in val_texts if text is not None]

/Users/ongsijing/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ongsijing/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-10 15:49:04.191381: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts
train_encodings = tokenizer(text=train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(text=val_texts, truncation=True, padding=True, max_length=256)

# Create a torch dataset
class SkinDiseaseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SkinDiseaseDataset(train_encodings, train_labels)
val_dataset = SkinDiseaseDataset(val_encodings, val_labels)

# Initialize BERT for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments and train
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

# Save the model and the label encoder for later use
model.save_pretrained("./skin_disease_model")
label_encoder.classes_.dump("./label_classes.pkl")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  8%|▊         | 10/126 [01:09<12:59,  6.72s/it]

{'loss': 1.3271, 'learning_rate': 4.603174603174603e-05, 'epoch': 0.24}


 16%|█▌        | 20/126 [02:21<12:10,  6.89s/it]

{'loss': 1.0831, 'learning_rate': 4.2063492063492065e-05, 'epoch': 0.48}


 24%|██▍       | 30/126 [03:31<11:22,  7.11s/it]

...

{'loss': 0.1975, 'learning_rate': 6.349206349206349e-06, 'epoch': 2.62}


 95%|█████████▌| 120/126 [17:04<00:52,  8.71s/it]

{'loss': 0.1877, 'learning_rate': 2.3809523809523808e-06, 'epoch': 2.86}


100%|██████████| 126/126 [17:48<00:00,  8.48s/it]


{'train_runtime': 1068.9303, 'train_samples_per_second': 0.929, 'train_steps_per_second': 0.118, 'train_loss': 0.6281599222667633, 'epoch': 3.0}


In [11]:
## Evaluation of training dataset

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test = pd.read_csv('test_symp.csv')
test_texts = test["Symptoms"].tolist()
test_labels = label_encoder.fit_transform(test["Disease"].tolist())

test_encodings = tokenizer(text=test_texts, truncation=True, padding=True, max_length=256)
test_dataset = SkinDiseaseDataset(test_encodings, test_labels)


# Predict using the trained model
predictions = trainer.predict(test_dataset)

# Get predicted labels
pred_labels = np.argmax(predictions.predictions, axis=1)

# Calculate metrics
accuracy = accuracy_score(test_labels, pred_labels)
precision = precision_score(test_labels, pred_labels, average='weighted')
recall = recall_score(test_labels, pred_labels, average='weighted')
f1 = f1_score(test_labels, pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


100%|██████████| 11/11 [00:20<00:00,  1.84s/it]

Accuracy: 0.8537
Precision: 0.8646
Recall: 0.8537
F1 Score: 0.8495


In [ ]:
## Classification model that uses BERT as the base, with a final layer for multi-class classification, corresponding to the various skin diseases.

from transformers import TFBertModel
import tensorflow as tf

# Load the pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define the maximum sequence length for your inputs. This will depend on your dataset.
max_length = 64  # You should adjust this value based on your actual data.

# The number of possible skin disease labels
num_labels = len(set(labels))  # 'labels' should be your list of diseases. Each disease is a possible label.

# Model construction: Input layers
input_ids_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_masks_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

# BERT layer: Extracting BERT's output
bert_output = bert_model(input_ids_layer, attention_mask=attention_masks_layer)[1]

# The output layer for classification
output_layer = tf.keras.layers.Dense(num_labels, activation='softmax')(bert_output)

# Combining everything into a Keras model
model = tf.keras.Model(inputs=[input_ids_layer, attention_masks_layer], outputs=output_layer)

# Compile the model with the optimizer, loss, and metrics you want to train with
model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Now, your model is ready for training with 'input_ids', 'attention_masks', and 'labels' (the actual diseases)


In [ ]:
## Evaluation of 

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# Assuming test_inputs and test_labels are your input and true labels for the test set
# and that your model has been trained with the name 'model'

test_input_ids, test_attention_masks = encode_texts(test_symptom_descriptions)  # use your encode_texts function

# Generate predictions
predictions = model.predict([test_input_ids, test_attention_masks])

# The predictions are in one-hot format (probabilities for each class), 
# so we'll convert them to labels
predicted_labels = np.argmax(predictions, axis=1)

# Assuming that your test_labels are also in a one-hot format, we need to convert them
true_labels = np.argmax(test_labels, axis=1)  # Remove this line if your labels are not one-hot encoded

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')  # use 'micro', 'macro', 'weighted', or 'samples'
recall = recall_score(true_labels, predicted_labels, average='weighted')  # use 'micro', 'macro', 'weighted', or 'samples'
f1 = f1_score(true_labels, predicted_labels, average='weighted')  # use 'micro', 'macro', 'weighted', or 'samples'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


# GPT model

In [8]:
from transformers import GPT2Tokenizer, TFGPT2ForSequenceClassification
import tensorflow as tf

# Load a pre-trained GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=num_diseases)

# Assume you have your dataset loaded in the `symptom_texts` and `labels` (disease names in string)
# The dataset is split into training and testing datasets

# Preparing the tokenized input for GPT-2 from your dataset of symptom descriptions
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert to TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels  # this should be numerical IDs, not the disease names in string
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# Training the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16), epochs=3, batch_size=16, validation_data=val_dataset.batch(16))

# After training, you can save the model
model.save_pretrained("./gpt_finetuned_skin_diseases/")


2023-11-03 22:33:57.328277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: 
TFGPT2ForSequenceClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.


# FastText

In [ ]:
# Assume 'combined_data' is a list of tuples (or similar) with the symptom description and the disease name
# Example: [("The skin is itchy...", "Eczema"), (...)]

with open("fasttext_train_data.txt", "w") as f:
    for description, disease in df:
        # Ensure the text is cleaned and normalized, if it's not already
        f.write(f"__label__{disease.replace(' ', '_')} {description}\n")  # FastText expects labels in this '__label__<class>' format.



In [ ]:
import fasttext

# Train the model
model = fasttext.train_supervised("fasttext_train_data.txt", epoch=25, wordNgrams=2)

# Saving the model
model.save_model("disease_prediction_model.ftz")


In [ ]:
# Predicting with the model
result = model.predict("The skin is red and itchy...")  # insert a real symptom description here

# 'result' will contain the labels and associated probabilities
disease_predicted = result[0][0].replace("__label__", "")  # we remove the label prefix to get the disease name
